# Calculate Vacancy Formation Energy in Silicon using MatterSim Potential

This notebook demonstrates how to compute the vacancy formation energy in silicon using the MatterSim potential within the Mat3ra framework.

## Methodology

The vacancy formation energy is calculated using the following approach:

1. **Create pristine supercell**: Build a silicon supercell from the unit cell and relax it using MatterSim potential
2. **Create vacancy**: Remove a single atom from the center of the supercell
3. **Relax vacancy structure**: Optimize the geometry of the defective supercell
4. **Calculate formation energy**: Compute the energy difference according to the formula:
   
   E_formation = E_vacancy - (N_vacancy/N_bulk) × E_bulk
   
   where:
   - E_vacancy: Total energy of the supercell with vacancy
   - E_bulk: Total energy of the pristine supercell
   - N_vacancy: Number of atoms in vacancy supercell
   - N_bulk: Number of atoms in pristine supercell

In [ ]:
# Install required packages if not already installed
# INFO: if not installed correctly, clear environment and run `pip install .[forcefields]` in the terminal
try:
    import mattersim
except ImportError:
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "pip", "install", ".[forcefields]", "--quiet"], check=False)
    import mattersim

## 1. Prepare materials
### 1.1. Load material data

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material.create(Materials.get_by_name_first_match("Si"))

### 1.2. Add vacancy to material

In [ ]:
from mat3ra.made.tools.helpers import create_vacancy, create_supercell

supercell = create_supercell(material, scaling_factor=[2,2,2])
material_with_vacancy = create_vacancy(supercell, coordinate=[0.5, 0.5, 0.5], placement_method="closest_site")

## 1.3. Visualize materials

In [ ]:
from utils.visualize import visualize_materials

visualize_materials([material, supercell, material_with_vacancy], rotation="-90x,-90y")

## 2. Setup calculation
### 2.1. Convert to ASE atoms

In [ ]:
from mat3ra.made.tools.convert import to_ase

material_atoms = to_ase(material)
material_with_vacancy_atoms = to_ase(material_with_vacancy)

### 2.2. Setup MatterSim calculator

In [ ]:
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

material_atoms.calc = MatterSimCalculator()
material_with_vacancy_atoms.calc = MatterSimCalculator()


### 2.3. Relax structures

In [ ]:
relaxer = Relaxer(optimizer="BFGS", constrain_symmetry=True)
relaxer.relax(material_atoms, steps=500)  # In-place relaxation
relaxer.relax(material_with_vacancy_atoms, steps=500)  # In-place relaxation


### 2.4. Visualize relaxed structures

In [ ]:
from mat3ra.made.tools.convert import from_ase

relaxed_material = Material.create(from_ase(material_atoms))
relaxed_material_with_vacancy = Material.create(from_ase(material_with_vacancy_atoms))

visualize_materials([relaxed_material, supercell, relaxed_material_with_vacancy], rotation="-90x,-90y")

## 3. Calculate vacancy formation energy

### 3.1. Get energies and atom counts

In [ ]:
energy_bulk = material_atoms.get_potential_energy()
n_atoms_bulk = len(material_atoms)
energy_vacancy = material_with_vacancy_atoms.get_potential_energy()
n_atoms_vac = len(material_with_vacancy_atoms)

### 3.2. Calculate vacancy formation energy

In [ ]:
e_vacancy = energy_vacancy - (n_atoms_vac / n_atoms_bulk * energy_bulk)
print(f"Vacancy formation energy: {e_vacancy:.4f} eV")
